In [1]:
import torch
from torch import nn
import torchvision as tv
from torchvision import datasets
import torch.onnx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from __future__ import print_function
from sklearn.datasets import fetch_openml
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

%matplotlib inline

In [2]:
batch_size = 12

train_data = datasets.MNIST('./mnist', train=True, download=True, transform=tv.transforms.ToTensor())
test_data = datasets.MNIST('./mnist', train=False, download=True, transform=tv.transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)


Extracting ./mnist\MNIST\raw\train-images-idx3-ubyte.gz to ./mnist\MNIST\raw




Extracting ./mnist\MNIST\raw\train-labels-idx1-ubyte.gz to ./mnist\MNIST\raw




Extracting ./mnist\MNIST\raw\t10k-images-idx3-ubyte.gz to ./mnist\MNIST\raw




Extracting ./mnist\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./mnist\MNIST\raw



In [ ]:
mnist = fetch_openml('mnist_784', version=1, cache=True)
X = mnist.data / 255.0
y = mnist.target

In [ ]:
feat_cols = [f"pixel{i}" for i in range(X.shape[1])]
df = pd.DataFrame(X, columns=feat_cols)
df['y'] = y
df

In [ ]:
np.random.seed(42)
rndperm = np.random.permutation(df.shape[0])

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        
        #Set dimensions
        self.in_dim = 28*28
        self.out_dim = 10
        
        #Set perceptrons
        self.fc1 = nn.Linear(self.in_dim, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64, self.out_dim)
        
        #Set activation functions
        self.relu = nn.ReLU()
        self.log_softmax = nn.LogSoftmax()
    
    #Set procedure
    def forward(self, x):
        a1 = self.relu(self.fc1(x.view(-1, self.in_dim)))
        a2 = self.relu(self.fc2(a1))
        a3 = self.relu(self.fc3(a2))
        a4 = self.relu(self.fc4(a3))
        logit = self.fc5(a4)
        
        return logit
    
    #Get outputs that passed layers for X
    def z1(self, x):
        return self.fc1(x.view(-1, self.in_dim))
    
    def a1(self, x):
        return self.relu(self.z1(x))
    
    def z2(self, x):
        return self.fc2(self.a1(x))
    
    def a2(self, x):
        return self.relu(self.z2(x))
    
    def z3(self, x):
        return self.fc3(self.a2(x))
    
    def a3(self, x):
        return self.relu(self.z3(x))
    
    def z4(self, x):
        return self.fc4(self.a3(x))
    
    def a4(self, x):
        return self.relu(self.z4(x))
    
    def z5(self, x):
        return self.fc5(self.a4(x))
    
    def a5(self, x):
        return self.log_softmax(self.z5(x), dim=-1)

In [ ]:
def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [ ]:
model = MLP()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [ ]:
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if (i+1) % 2000 == 0:
            print(f"[{epoch+1}, {i+1}] loss : {running_loss/2000}")
            running_loss = 0.0
print("Finish")

In [ ]:
dataiter = iter(test_loader)
images, labels = dataiter.next()

imshow(tv.utils.make_grid(images, nrow=batch_size))
print('GroundTruth')
for label in labels:
    print(label.item(), end=' ')

outputs = model(images)
_, predicted = torch.max(outputs, 1)
print('\nPrediction')
for label in predicted:
    print(label.item(), end=' ')

In [ ]:
n_predict = 0
n_correct = 0

for data in test_loader:
    inputs, labels = data
    outputs = model(inputs)
    _, predicted = torch.max(outputs, 1)
    
    n_predict += len(predicted)
    n_correct += (labels == predicted).sum()
    
print(f"{n_correct}/{n_predict}")
print(f"Accuracy : {n_correct/n_predict:.3f}")

# PCA

In [ ]:
pca = PCA(n_components=2)
pca_result = pca.fit_transform(df[feat_cols].values)
df['pca-one'] = pca_result[:, 0]
df['pca-two'] = pca_result[:, 1]

# t-SNE

In [ ]:
N = 10000
df_subset = df.loc[rndperm[:N], :].copy()
data_subset = df_subset[feat_cols].values

tsne = TSNE(n_components=2, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(data_subset)
df_subset['tsne-2d-one'] = tsne_results[:, 0]
df_subset['tsne-2d-two'] = tsne_results[:, 1]

# z[1] PCA

In [ ]:
z1 = model.z1(torch.from_numpy(df[feat_cols].values).float())
z1 = z1.detach().numpy()

pca_z1 = PCA(n_components=2)
pca_z1_result = pca_z1.fit_transform(z1)
df['pca-z1-one'] = pca_z1_result[:, 0]
df['pca-z1-two'] = pca_z1_result[:, 1]

# z[1] t-SNE

In [ ]:
df_z1_subset = pd.concat([pd.DataFrame(z1[rndperm[:N], :], index=rndperm[:N]), df.loc[rndperm[:N], :]['y']], axis=1)
data_z1_subset = z1[rndperm[:N], :]

tsne_z1 = TSNE(n_components=2, perplexity=40, n_iter=300)
tsne_z1_results = tsne_z1.fit_transform(data_z1_subset)
df_z1_subset['tsne-z1-2d-one'] = tsne_z1_results[:, 0]
df_z1_subset['tsne-z1-2d-two'] = tsne_z1_results[:, 1]

# a[1] PCA

In [ ]:
a1 = model.a1(torch.from_numpy(df[feat_cols].values).float())
a1 = a1.detach().numpy()

pca_a1 = PCA(n_components=2)
pca_a1_result = pca_a1.fit_transform(a1)
df['pca-a1-one'] = pca_a1_result[:, 0]
df['pca-a1-two'] = pca_a1_result[:, 1]

# a[1] t-SNE

In [ ]:
df_a1_subset = pd.concat([pd.DataFrame(a1[rndperm[:N], :], index=rndperm[:N]), df.loc[rndperm[:N], :]['y']], axis=1)
data_a1_subset = a1[rndperm[:N], :]

tsne_a1 = TSNE(n_components=2, perplexity=40, n_iter=300)
tsne_a1_results = tsne_a1.fit_transform(data_a1_subset)
df_a1_subset['tsne-a1-2d-one'] = tsne_a1_results[:, 0]
df_a1_subset['tsne-a1-2d-two'] = tsne_a1_results[:, 1]

# z[2] PCA

In [ ]:
z2 = model.z2(torch.from_numpy(df[feat_cols].values).float())
z2 = z2.detach().numpy()

pca_z2 = PCA(n_components=2)
pca_z2_result = pca_z2.fit_transform(z2)
df['pca-z2-one'] = pca_z2_result[:, 0]
df['pca-z2-two'] = pca_z2_result[:, 1]

# z[2] t-SNE

In [ ]:
df_z2_subset = pd.concat([pd.DataFrame(z2[rndperm[:N], :], index=rndperm[:N]), df.loc[rndperm[:N], :]['y']], axis=1)
data_z2_subset = z2[rndperm[:N], :]

tsne_z2 = TSNE(n_components=2, perplexity=40, n_iter=300)
tsne_z2_results = tsne_z2.fit_transform(data_z2_subset)
df_z2_subset['tsne-z2-2d-one'] = tsne_z2_results[:, 0]
df_z2_subset['tsne-z2-2d-two'] = tsne_z2_results[:, 1]

# a[2] PCA

In [ ]:
a2 = model.a2(torch.from_numpy(df[feat_cols].values).float())
a2 = a2.detach().numpy()

pca_a2 = PCA(n_components=2)
pca_a2_result = pca_a2.fit_transform(a2)
df['pca-a2-one'] = pca_a2_result[:, 0]
df['pca-a2-two'] = pca_a2_result[:, 1]

# a[2] t-SNE

In [ ]:
df_a2_subset = pd.concat([pd.DataFrame(a2[rndperm[:N], :], index=rndperm[:N]), df.loc[rndperm[:N], :]['y']], axis=1)
data_a2_subset = a2[rndperm[:N], :]

tsne_a2 = TSNE(n_components=2, perplexity=40, n_iter=300)
tsne_a2_results = tsne_a2.fit_transform(data_a2_subset)
df_a2_subset['tsne-a2-2d-one'] = tsne_a2_results[:, 0]
df_a2_subset['tsne-a2-2d-two'] = tsne_a2_results[:, 1]

# z[3] PCA

In [ ]:
z3 = model.z3(torch.from_numpy(df[feat_cols].values).float())
z3 = z3.detach().numpy()

pca_z3 = PCA(n_components=2)
pca_z3_result = pca_z3.fit_transform(z3)
df['pca-z3-one'] = pca_z3_result[:, 0]
df['pca-z3-two'] = pca_z3_result[:, 1]

# z[3] t-SNE

In [ ]:
df_z3_subset = pd.concat([pd.DataFrame(z3[rndperm[:N], :], index=rndperm[:N]), df.loc[rndperm[:N], :]['y']], axis=1)
data_z3_subset = z3[rndperm[:N], :]

tsne_z3 = TSNE(n_components=2, perplexity=40, n_iter=300)
tsne_z3_results = tsne_z3.fit_transform(data_z3_subset)
df_z3_subset['tsne-z3-2d-one'] = tsne_z3_results[:, 0]
df_z3_subset['tsne-z3-2d-two'] = tsne_z3_results[:, 1]

# a[3] PCA

In [ ]:
a3 = model.a3(torch.from_numpy(df[feat_cols].values).float())
a3 = a3.detach().numpy()

pca_a3 = PCA(n_components=2)
pca_a3_result = pca_a3.fit_transform(a3)
df['pca-a3-one'] = pca_a3_result[:, 0]
df['pca-a3-two'] = pca_a3_result[:, 1]

# a[3] t-SNE

In [ ]:
df_a3_subset = pd.concat([pd.DataFrame(a3[rndperm[:N], :], index=rndperm[:N]), df.loc[rndperm[:N], :]['y']], axis=1)
data_a3_subset = a3[rndperm[:N], :]

tsne_a3 = TSNE(n_components=2, perplexity=40, n_iter=300)
tsne_a3_results = tsne_a3.fit_transform(data_a3_subset)
df_a3_subset['tsne-a3-2d-one'] = tsne_a3_results[:, 0]
df_a3_subset['tsne-a3-2d-two'] = tsne_a3_results[:, 1]

# z[4] PCA

In [ ]:
z4 = model.z4(torch.from_numpy(df[feat_cols].values).float())
z4 = z4.detach().numpy()

pca_z4 = PCA(n_components=2)
pca_z4_result = pca_z4.fit_transform(z4)
df['pca-z4-one'] = pca_z4_result[:, 0]
df['pca-z4-two'] = pca_z4_result[:, 1]

# z[4] t-SNE

In [ ]:
df_z4_subset = pd.concat([pd.DataFrame(z4[rndperm[:N], :], index=rndperm[:N]), df.loc[rndperm[:N], :]['y']], axis=1)
data_z4_subset = z4[rndperm[:N], :]

tsne_z4 = TSNE(n_components=2, perplexity=40, n_iter=300)
tsne_z4_results = tsne_z4.fit_transform(data_z4_subset)
df_z4_subset['tsne-z4-2d-one'] = tsne_z4_results[:, 0]
df_z4_subset['tsne-z4-2d-two'] = tsne_z4_results[:, 1]

# a[4] PCA

In [ ]:
a4 = model.a4(torch.from_numpy(df[feat_cols].values).float())
a4 = a4.detach().numpy()

pca_a4 = PCA(n_components=2)
pca_a4_result = pca_a4.fit_transform(a4)
df['pca-a4-one'] = pca_a4_result[:, 0]
df['pca-a4-two'] = pca_a4_result[:, 1]

# a[4] t-SNE

In [ ]:
df_a4_subset = pd.concat([pd.DataFrame(a4[rndperm[:N], :], index=rndperm[:N]), df.loc[rndperm[:N], :]['y']], axis=1)
data_a4_subset = a4[rndperm[:N], :]

tsne_a4 = TSNE(n_components=2, perplexity=40, n_iter=300)
tsne_a4_results = tsne_a4.fit_transform(data_a4_subset)
df_a4_subset['tsne-a4-2d-one'] = tsne_a4_results[:, 0]
df_a4_subset['tsne-a4-2d-two'] = tsne_a4_results[:, 1]

# z[5] PCA

In [ ]:
z5 = model.z5(torch.from_numpy(df[feat_cols].values).float())
z5 = z5.detach().numpy()

pca_z5 = PCA(n_components=2)
pca_z5_result = pca_z5.fit_transform(z5)
df['pca-z5-one'] = pca_z5_result[:, 0]
df['pca-z5-two'] = pca_z5_result[:, 1]

# z[5] t-SNE

In [ ]:
df_z5_subset = pd.concat([pd.DataFrame(z5[rndperm[:N], :], index=rndperm[:N]), df.loc[rndperm[:N], :]['y']], axis=1)
data_z5_subset = z5[rndperm[:N], :]

tsne_z5 = TSNE(n_components=2, perplexity=40, n_iter=300)
tsne_z5_results = tsne_z5.fit_transform(data_z5_subset)
df_z5_subset['tsne-z5-2d-one'] = tsne_z5_results[:, 0]
df_z5_subset['tsne-z5-2d-two'] = tsne_z5_results[:, 1]

# a[5] PCA

In [ ]:
a5 = model.a5(torch.from_numpy(df[feat_cols].values).float())
a5 = a5.detach().numpy()

pca_a5 = PCA(n_components=2)
pca_a5_result = pca_a5.fit_transform(a5)
df['pca-a5-one'] = pca_a5_result[:, 0]
df['pca-a5-two'] = pca_a5_result[:, 1]

# a[5] t-SNE

In [ ]:
df_a5_subset = pd.concat([pd.DataFrame(a5[rndperm[:N], :], index=rndperm[:N]), df.loc[rndperm[:N], :]['y']], axis=1)
data_a5_subset = a5[rndperm[:N], :]

tsne_a5 = TSNE(n_components=2, perplexity=40, n_iter=300)
tsne_a5_results = tsne_a5.fit_transform(data_a5_subset)
df_a5_subset['tsne-a5-2d-one'] = tsne_a5_results[:, 0]
df_a5_subset['tsne-a5-2d-two'] = tsne_a5_results[:, 1]

In [ ]:
plt.figure(figsize=(7, 7))

fig, line = plt.subplots(11, 2, figsize=(15,80))
sns.set_theme(font_scale=1.5)

sns.scatterplot(x='pca-one', y='pca-two', hue='y', palette=sns.color_palette("hls", 10), data=df.loc[rndperm,:], legend='full', alpha=0.3, ax=line[0][0])
sns.scatterplot(x='tsne-2d-one', y='tsne-2d-two', hue='y', palette=sns.color_palette("hls", 10), data=df_subset, legend='full', alpha=0.3, ax=line[0][1])
sns.scatterplot(x='pca-z1-one', y='pca-z1-two', hue='y', palette=sns.color_palette("hls", 10), data=df.loc[rndperm,:], legend='full', alpha=0.3, ax=line[1][0])
sns.scatterplot(x='tsne-z1-2d-one', y='tsne-z1-2d-two', hue='y', palette=sns.color_palette("hls", 10), data=df_z1_subset, legend='full', alpha=0.3, ax=line[1][1])
sns.scatterplot(x='pca-a1-one', y='pca-a1-two', hue='y', palette=sns.color_palette("hls", 10), data=df.loc[rndperm,:], legend='full', alpha=0.3, ax=line[2][0])
sns.scatterplot(x='tsne-a1-2d-one', y='tsne-a1-2d-two', hue='y', palette=sns.color_palette("hls", 10), data=df_a1_subset, legend='full', alpha=0.3, ax=line[2][1])
sns.scatterplot(x='pca-z2-one', y='pca-z2-two', hue='y', palette=sns.color_palette("hls", 10), data=df.loc[rndperm,:], legend='full', alpha=0.3, ax=line[3][0])
sns.scatterplot(x='tsne-z2-2d-one', y='tsne-z2-2d-two', hue='y', palette=sns.color_palette("hls", 10), data=df_z2_subset, legend='full', alpha=0.3, ax=line[3][1])
sns.scatterplot(x='pca-a2-one', y='pca-a2-two', hue='y', palette=sns.color_palette("hls", 10), data=df.loc[rndperm,:], legend='full', alpha=0.3, ax=line[4][0])
sns.scatterplot(x='tsne-a2-2d-one', y='tsne-a2-2d-two', hue='y', palette=sns.color_palette("hls", 10), data=df_a2_subset, legend='full', alpha=0.3, ax=line[4][1])
sns.scatterplot(x='pca-z3-one', y='pca-z3-two', hue='y', palette=sns.color_palette("hls", 10), data=df.loc[rndperm,:], legend='full', alpha=0.3, ax=line[5][0])
sns.scatterplot(x='tsne-z3-2d-one', y='tsne-z3-2d-two', hue='y', palette=sns.color_palette("hls", 10), data=df_z3_subset, legend='full', alpha=0.3, ax=line[5][1])
sns.scatterplot(x='pca-a3-one', y='pca-a3-two', hue='y', palette=sns.color_palette("hls", 10), data=df.loc[rndperm,:], legend='full', alpha=0.3, ax=line[6][0])
sns.scatterplot(x='tsne-a3-2d-one', y='tsne-a3-2d-two', hue='y', palette=sns.color_palette("hls", 10), data=df_a3_subset, legend='full', alpha=0.3, ax=line[6][1])
sns.scatterplot(x='pca-z4-one', y='pca-z4-two', hue='y', palette=sns.color_palette("hls", 10), data=df.loc[rndperm,:], legend='full', alpha=0.3, ax=line[7][0])
sns.scatterplot(x='tsne-z4-2d-one', y='tsne-z4-2d-two', hue='y', palette=sns.color_palette("hls", 10), data=df_z4_subset, legend='full', alpha=0.3, ax=line[7][1])
sns.scatterplot(x='pca-a4-one', y='pca-a4-two', hue='y', palette=sns.color_palette("hls", 10), data=df.loc[rndperm,:], legend='full', alpha=0.3, ax=line[8][0])
sns.scatterplot(x='tsne-a4-2d-one', y='tsne-a4-2d-two', hue='y', palette=sns.color_palette("hls", 10), data=df_a4_subset, legend='full', alpha=0.3, ax=line[8][1])
sns.scatterplot(x='pca-z5-one', y='pca-z5-two', hue='y', palette=sns.color_palette("hls", 10), data=df.loc[rndperm,:], legend='full', alpha=0.3, ax=line[9][0])
sns.scatterplot(x='tsne-z5-2d-one', y='tsne-z5-2d-two', hue='y', palette=sns.color_palette("hls", 10), data=df_z5_subset, legend='full', alpha=0.3, ax=line[9][1])
sns.scatterplot(x='pca-a5-one', y='pca-a5-two', hue='y', palette=sns.color_palette("hls", 10), data=df.loc[rndperm,:], legend='full', alpha=0.3, ax=line[10][0])
sns.scatterplot(x='tsne-a5-2d-one', y='tsne-a5-2d-two', hue='y', palette=sns.color_palette("hls", 10), data=df_a5_subset, legend='full', alpha=0.3, ax=line[10][1])

line[0][0].set_title("a[0] PCA", fontsize=24)
line[0][1].set_title("a[0] t-SNE", fontsize=24)
line[1][0].set_title("z[1] PCA", fontsize=24)
line[1][1].set_title("z[1] t-SNE", fontsize=24)
line[2][0].set_title("a[1] PCA", fontsize=24)
line[2][1].set_title("a[1] t-SNE", fontsize=24)
line[3][0].set_title("z[2] PCA", fontsize=24)
line[3][1].set_title("z[2] t-SNE", fontsize=24)
line[4][0].set_title("a[2] PCA", fontsize=24)
line[4][1].set_title("a[2] t-SNE", fontsize=24)
line[5][0].set_title("z[3] PCA", fontsize=24)
line[5][1].set_title("z[3] t-SNE", fontsize=24)
line[6][0].set_title("a[3] PCA", fontsize=24)
line[6][1].set_title("a[3] t-SNE", fontsize=24)
line[7][0].set_title("z[4] PCA", fontsize=24)
line[7][1].set_title("z[4] t-SNE", fontsize=24)
line[8][0].set_title("a[4] PCA", fontsize=24)
line[8][1].set_title("a[4] t-SNE", fontsize=24)
line[9][0].set_title("z[5] PCA", fontsize=24)
line[9][1].set_title("z[5] t-SNE", fontsize=24)
line[10][0].set_title("a[5] PCA", fontsize=24)
line[10][1].set_title("a[5] t-SNE", fontsize=24)

fig.tight_layout(pad=3)

plt.show()